In [1]:
!ls

sample_data


In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!curl --header "Host: launchpadlibrarian.net" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header "Accept-Language: zh,zh-CN;q=0.9,zh-TW;q=0.8,en-US;q=0.7,en;q=0.6" "https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb" -o "google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb" -L
!apt-get -y install -qq fuse
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1203k  100 1203k    0     0  6204k      0 --:--:-- --:--:-- --:--:-- 6172k
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 22306 files and directories currently installed.)
Preparing to unpack google-drive-ocamlfuse_0

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
!ls drive > /dev/null

### Setup

In [2]:
!mkdir src -p && cd src && git clone https://github.com/fastai/fastai.git

Cloning into 'fastai'...
remote: Enumerating objects: 13467, done.
remote: Total 13467 (delta 0), reused 0 (delta 0), pack-reused 13467
Receiving objects: 100% (13467/13467), 248.80 MiB | 26.51 MiB/s, done.
Resolving deltas: 100% (8770/8770), done.
Checking out files: 100% (681/681), done.


In [3]:
!pip3 install -q bcolz graphviz sklearn_pandas isoweek pandas_summary ipywidgets torch torchvision torchtext

tcmalloc: large alloc 1073750016 bytes == 0x59754000 @  0x7f35dc5102a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
!git config --global user.email 'nikshuang@163.com'
!git config --global user.name 'Alex Huang'
!pip install -q kaggle

In [5]:
import sys
sys.path.append("/content/src/fastai/old") # on windows use \'s instead

from fastai.structured import *
from fastai.column_data import *
from itertools import product
np.set_printoptions(threshold=50, edgeitems=20)
import gc
print(torch.__version__)
torch.backends.cudnn.benchmark=True
torch.cuda.is_available(), torch.backends.cudnn.enabled

0.4.1


(True, True)

In [0]:
PATH = Path('data/predict_feature_sales')
DISK_PATH = Path('drive/predict_feature_sales')
os.makedirs(PATH, exist_ok=True)
os.makedirs(DISK_PATH, exist_ok=True)

In [8]:
!kaggle competitions download -c competitive-data-science-predict-future-sales -p {PATH}

  0% 0.00/3.49k [00:00<?, ?B/s]
100% 3.49k/3.49k [00:00<00:00, 2.64MB/s]
  0% 0.00/1.50M [00:00<?, ?B/s]
100% 1.50M/1.50M [00:00<00:00, 103MB/s]
  0% 0.00/2.91k [00:00<?, ?B/s]
100% 2.91k/2.91k [00:00<00:00, 3.62MB/s]
  0% 0.00/469k [00:00<?, ?B/s]
100% 469k/469k [00:00<00:00, 81.0MB/s]
  0% 0.00/1.01M [00:00<?, ?B/s]
100% 1.01M/1.01M [00:00<00:00, 122MB/s]
 70% 9.00M/12.9M [00:00<00:00, 20.1MB/s]
100% 12.9M/12.9M [00:00<00:00, 31.2MB/s]


In [9]:
!cd {PATH} && ls *.gz | xargs -IX gunzip X
!ls {PATH}

item_categories.csv  sales_train.csv	    shops.csv
items.csv	     sample_submission.csv  test.csv


### Data Cleaning

In [0]:
train = pd.read_csv(PATH/'sales_train.csv', low_memory=False)
test = pd.read_csv(PATH/'test.csv', low_memory=False).set_index('ID')
shops = pd.read_csv(PATH/'shops.csv', low_memory=False)
items = pd.read_csv(PATH/'items.csv', low_memory=False)
cats = pd.read_csv(PATH/'item_categories.csv', low_memory=False)

In [61]:
train.head(3)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,02.01.2013,0,59,22154,999.0,1.0,999.0
1,03.01.2013,0,25,2552,899.0,1.0,899.0
2,05.01.2013,0,25,2552,899.0,-1.0,-899.0


In [62]:
test.head(2)

,shop_id,item_id
ID,,
0,5,5037
1,5,5320


In [0]:
# train.date = pd.to_datetime(train.date, format='%d.%m.%Y')
train = train[train.item_cnt_day <= 1000]
train = train[train.item_price < 100000]
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

In [0]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

### Feature Engineering

In [0]:
def join_df_agg(left, right, aggregation_recipies, col_names=None, col_type=None):
  groupby_aggregate_names = []
  if col_names is not None:
    col_names.reverse()
  for i, (groupby_cols, specs) in enumerate(aggregation_recipies):
      group_object = right.groupby(groupby_cols)
      for select, agg in specs:
          if col_names is not None:
            groupby_aggregate_name = col_names.pop()
          else:
            groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
          left = left.merge(group_object[select]
                                .agg(agg)
                                .reset_index()
                                .rename(index=str,
                                        columns={select: groupby_aggregate_name})
                                [groupby_cols + [groupby_aggregate_name]],
                                on=groupby_cols,
                                how='left')
          if col_type: left[groupby_aggregate_name] = left[groupby_aggregate_name].astype(col_type)
          groupby_aggregate_names.append(groupby_aggregate_name)
  return left, groupby_aggregate_names

In [0]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

In [0]:
recipies = [
    (['date_block_num', 'shop_id', 'item_id'], [('item_cnt_day', 'sum')]),
    (['item_id'], [('item_price', 'mean')]),
    (['date_block_num', 'item_id'], [('item_price', 'mean')]),
    (['date_block_num','shop_id'], [('revenue', 'sum')]),
]
col_names = [
    'item_cnt_month', 'item_avg_item_price', 'date_item_avg_item_price',
    'date_shop_revenue'
]

matrix, _ = join_df_agg(train, train, recipies, col_names)
matrix.item_cnt_month = matrix.item_cnt_month.fillna(0).clip(0, 20)
                  
recipies = [
    (['shop_id'], [('date_shop_revenue', 'mean')]),
]
col_names = [
    'shop_avg_revenue'
]
matrix, _ = join_df_agg(matrix, matrix, recipies, col_names)
matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']

In [0]:
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')

In [0]:
matrix.drop(['date', 'item_price', 'item_cnt_day', 'revenue'], axis=1, inplace=True)
matrix.drop_duplicates(inplace=True)

In [0]:
recipies = [
    (['date_block_num'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'item_id'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'shop_id'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'item_category_id'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'shop_id', 'item_category_id'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'shop_id', 'type_code'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'shop_id', 'subtype_code'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'city_code'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'item_id', 'city_code'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'type_code'], [('item_cnt_month', 'mean')]),
    (['date_block_num', 'subtype_code'], [('item_cnt_month', 'mean')]),
]
col_names = [
    'date_avg_item_cnt', 'date_item_avg_item_cnt', 'date_shop_avg_item_cnt', 'date_cat_avg_item_cnt', 
    'date_shop_cat_avg_item_cnt', 'date_shop_type_avg_item_cnt', 'date_shop_subtype_avg_item_cnt',
    'date_city_avg_item_cnt', 'date_item_city_avg_item_cnt', 'date_type_avg_item_cnt', 'date_subtype_avg_item_cnt'
]

matrix, _ = join_df_agg(matrix, matrix, recipies, col_names)

In [0]:
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)

In [28]:
%%time
item_shop_cache = {}
item_cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_last_sale'] = -1

for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in item_shop_cache:
        if row.item_cnt_month!=0:
            item_shop_cache[key] = row.date_block_num
    else:
        last_date_block_num = item_shop_cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        item_shop_cache[key] = row.date_block_num   
   
    key = row.item_id
    if key not in item_cache:
        if row.item_cnt_month!=0:
            item_cache[key] = row.date_block_num
    else:
        last_date_block_num = item_cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            item_cache[key] = row.date_block_num

matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

CPU times: user 3min 38s, sys: 365 ms, total: 3min 38s
Wall time: 3min 38s


In [29]:
matrix.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_avg_item_price', 'date_item_avg_item_price', 'date_shop_revenue',
       'shop_avg_revenue', 'delta_revenue', 'city_code', 'item_category_id',
       'type_code', 'subtype_code', 'date_avg_item_cnt',
       'date_item_avg_item_cnt', 'date_shop_avg_item_cnt',
       'date_cat_avg_item_cnt', 'date_shop_cat_avg_item_cnt',
       'date_shop_type_avg_item_cnt', 'date_shop_subtype_avg_item_cnt',
       'date_city_avg_item_cnt', 'date_item_city_avg_item_cnt',
       'date_type_avg_item_cnt', 'date_subtype_avg_item_cnt', 'month', 'days',
       'item_shop_last_sale', 'item_last_sale', 'item_shop_first_sale',
       'item_first_sale'],
      dtype='object')

In [30]:
del items, shops, cats
gc.collect()

121

In [31]:
matrix.reset_index(drop=True, inplace=True)
matrix.to_pickle('matrix.pkl')
shutil.copy('matrix.pkl', DISK_PATH)

'drive/predict_feature_sales/matrix.pkl'

### Training

In [12]:
shutil.copy(DISK_PATH/'matrix.pkl', '.')

'./matrix.pkl'

In [0]:
trn_df = pickle.load(open('matrix.pkl', 'rb'))

In [0]:
trn_df = matrix

In [33]:
trn_df.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_avg_item_price', 'date_item_avg_item_price', 'date_shop_revenue',
       'shop_avg_revenue', 'delta_revenue', 'city_code', 'item_category_id',
       'type_code', 'subtype_code', 'date_avg_item_cnt',
       'date_item_avg_item_cnt', 'date_shop_avg_item_cnt',
       'date_cat_avg_item_cnt', 'date_shop_cat_avg_item_cnt',
       'date_shop_type_avg_item_cnt', 'date_shop_subtype_avg_item_cnt',
       'date_city_avg_item_cnt', 'date_item_city_avg_item_cnt',
       'date_type_avg_item_cnt', 'date_subtype_avg_item_cnt', 'month', 'days',
       'item_shop_last_sale', 'item_last_sale', 'item_shop_first_sale',
       'item_first_sale'],
      dtype='object')

In [0]:
cont_cols = list(trn_df.columns)
cat_cols = ['date_block_num', 'shop_id', 'item_id', 'city_code',
       'item_category_id', 'type_code', 'subtype_code', 'month', 'days']
# cat_cols = ['date_block_num', 'shop_id', 'city_code',
#        'item_category_id', 'type_code', 'subtype_code', 'month', 'days']
y_col = 'item_cnt_month'
# skip_cols = ['item_id']
skip_cols = []
for o in cat_cols + [y_col] + skip_cols:
    cont_cols.remove(o)

In [0]:
for col in cat_cols: trn_df[col] = trn_df[col].astype('category').cat.as_ordered()
for col in cont_cols: trn_df[col] = trn_df[col].fillna(0).astype('float32')

In [36]:
cat_sz = [(o, len(trn_df[o].cat.categories) + 1) for o in cat_cols]
cat_sz

[('date_block_num', 35),
 ('shop_id', 58),
 ('item_id', 21807),
 ('city_code', 32),
 ('item_category_id', 85),
 ('type_code', 21),
 ('subtype_code', 66),
 ('month', 13),
 ('days', 4)]

In [37]:
emb_szs = [(o, min(50, o // 2)) for _, o in cat_sz]
emb_szs

[(35, 17),
 (58, 29),
 (21807, 50),
 (32, 16),
 (85, 42),
 (21, 10),
 (66, 33),
 (13, 6),
 (4, 2)]

In [0]:
df_trn, y, nas, mapper = proc_df(trn_df, 'item_cnt_month', skip_flds=skip_cols, do_scale=True)

In [39]:
n = len(df_trn); n

1609123

In [40]:
train_ratio = 0.75
train_size = int(n * train_ratio)
val_idxs = list(range(train_size, n))
len(val_idxs), np.array(val_idxs)

(402281,
 array([1206842, 1206843, 1206844, 1206845, 1206846, 1206847, 1206848, 1206849, 1206850, 1206851, 1206852,
        1206853, 1206854, 1206855, 1206856, 1206857, 1206858, 1206859, 1206860, 1206861, ..., 1609103,
        1609104, 1609105, 1609106, 1609107, 1609108, 1609109, 1609110, 1609111, 1609112, 1609113, 1609114,
        1609115, 1609116, 1609117, 1609118, 1609119, 1609120, 1609121, 1609122]))

In [0]:
bs = 128
md = ColumnarModelData.from_data_frame(PATH, val_idxs, df_trn, y.astype('float32'), cat_cols, bs=bs)

In [0]:
emb_drop = 0.04 * 2
szs = [1000, 500]
ps = np.array([0.25, 0.5])
learn = md.get_learner(emb_szs, len(cont_cols), emb_drop, 1, szs=szs, drops=ps, y_range=(0, 20))

In [58]:
learn

MixedInputModel(
  (embs): ModuleList(
    (0): Embedding(35, 17)
    (1): Embedding(58, 29)
    (2): Embedding(21807, 50)
    (3): Embedding(32, 16)
    (4): Embedding(85, 42)
    (5): Embedding(21, 10)
    (6): Embedding(66, 33)
    (7): Embedding(13, 6)
    (8): Embedding(4, 2)
  )
  (lins): ModuleList(
    (0): Linear(in_features=225, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=500, bias=True)
  )
  (bns): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (outp): Linear(in_features=500, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.08)
  (drops): ModuleList(
    (0): Dropout(p=0.25)
    (1): Dropout(p=0.5)
  )
  (bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [53]:
lr = 1e-3
learn.fit(lr, 1)

  0%|          | 11/9429 [00:00<13:59, 11.21it/s, loss=14.4]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch      trn_loss   val_loss   
    0      0.723625   0.677875  



[0.6778747571599526]

In [54]:
learn.fit(lr / 3, 3)

  0%|          | 11/9429 [00:00<12:55, 12.15it/s, loss=0.605]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch      trn_loss   val_loss   
    0      0.604653   0.634174  
    1      0.621239   0.638807  
    2      0.62528    0.631941  



[0.6319414914829438]

In [56]:
learn.crit

<function torch.nn.functional.mse_loss>

In [55]:
np.sqrt(0.631941)

0.79494716805584